# LMI Platform - Data Engineering & ETL

This notebook handles:
1. **Data Exploration** - Analyze CSV structure and quality
2. **Data Cleaning** - Handle missing values, duplicates, inconsistencies
3. **Data Transformation** - Normalize skills, parse languages, categorize data
4. **Data Seeding** - Load cleaned data into PostgreSQL database

---

## Setup & Imports

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import sys
import os

# Add parent directory to path to import models
sys.path.append(os.path.dirname(os.path.abspath('__file__')))

from app import app
from models import db, Company, Job, Skill, University, Program, SkillType

print("Imports successful")

Imports successful


## 1. Data Exploration

### 1.1 Load Jobs Dataset

In [2]:
# Load jobs data
jobs_df = pd.read_csv('data/cambodia_jobs_10k.csv')

print(f"Jobs Dataset Shape: {jobs_df.shape}")
print(f"\nColumns: {list(jobs_df.columns)}")
print(f"\nFirst 3 rows:")
jobs_df.head(3)

Jobs Dataset Shape: (10000, 15)

Columns: ['job_id', 'job_title', 'company_name', 'industry', 'location', 'employment_type', 'experience_level', 'salary_min_usd', 'salary_max_usd', 'required_skills', 'languages_required', 'degree_required', 'posted_date', 'is_active', 'description']

First 3 rows:


,job_id,job_title,company_name,industry,location,employment_type,experience_level,salary_min_usd,salary_max_usd,required_skills,languages_required,degree_required,posted_date,is_active,description
0,JOB00001,Site Engineer,China State Construction Engineering Corporation,Construction,Sihanoukville,Full-time,Senior,1250,3414,"Driving License, SQL, JavaScript",Khmer,Diploma,2025-11-10,False,We are seeking a talented Site Engineer to joi...
1,JOB00002,Marketing Coordinator,Chip Mong Land,Real Estate,Battambang,Internship,Mid Level,958,2059,"Python, PowerPoint, Driving License, SEO, Time...","Khmer, English",Diploma,2025-09-03,True,We are seeking a talented Marketing Coordinato...
2,JOB00003,Event Coordinator,Sofitel Phnom Penh Phokeethra,Hospitality/Tourism,Kampong Chhnang,Contract,Entry Level,241,852,"Khmer, Node.js, Adobe Illustrator",Khmer,PhD,2025-11-02,False,We are seeking a talented Event Coordinator to...


In [3]:
# Check data quality
print("Missing Values:")
print(jobs_df.isnull().sum())

print("\nData Types:")
print(jobs_df.dtypes)

print("\nBasic Statistics:")
jobs_df.describe()

Missing Values:
job_id                   0
job_title                0
company_name             0
industry                 0
location                 0
employment_type          0
experience_level         0
salary_min_usd           0
salary_max_usd           0
required_skills          0
languages_required       0
degree_required       1674
posted_date              0
is_active                0
description              0
dtype: int64

Data Types:
job_id                object
job_title             object
company_name          object
industry              object
location              object
employment_type       object
experience_level      object
salary_min_usd         int64
salary_max_usd         int64
required_skills       object
languages_required    object
degree_required       object
posted_date           object
is_active               bool
description           object
dtype: object

Basic Statistics:


,salary_min_usd,salary_max_usd
count,10000.000000,10000.000000
mean,1308.044800,3472.723600
std,989.834979,2668.661066
min,150.000000,500.000000
25%,397.000000,1193.750000
50%,997.000000,2484.500000
75%,2010.000000,5042.250000
max,3500.000000,9999.000000


### 1.2 Load Universities Dataset

In [4]:
# Load universities data
universities_df = pd.read_csv('data/cambodia_universities.csv')

print(f"Universities Dataset Shape: {universities_df.shape}")
print(f"\nColumns: {list(universities_df.columns)}")
print(f"\nFirst 3 rows:")
universities_df.head(3)

Universities Dataset Shape: (170, 14)

Columns: ['university_id', 'university_name', 'university_type', 'location', 'program_category', 'program_name', 'degree_level', 'duration_years', 'annual_tuition_usd', 'enrollment_capacity', 'accredited', 'languages_of_instruction', 'established_year', 'in_demand_field']

First 3 rows:


,university_id,university_name,university_type,location,program_category,program_name,degree_level,duration_years,annual_tuition_usd,enrollment_capacity,accredited,languages_of_instruction,established_year,in_demand_field
0,UNI0001,Royal University of Phnom Penh,Public,Phnom Penh,Engineering,Industrial Engineering,Bachelor,4,602,266,True,Khmer,1965,True
1,UNI0002,Royal University of Phnom Penh,Public,Phnom Penh,Education,Secondary Education,Bachelor,4,587,151,False,"Khmer, English",1974,False
2,UNI0003,Royal University of Phnom Penh,Public,Phnom Penh,Healthcare,Public Health,Bachelor,4,657,266,True,"Khmer, English",1969,True


In [5]:
# Check data quality
print("Missing Values:")
print(universities_df.isnull().sum())

print("\nUnique Universities:")
print(universities_df['university_name'].nunique())

print("\nProgram Categories:")
print(universities_df['program_category'].value_counts())

Missing Values:
university_id               0
university_name             0
university_type             0
location                    0
program_category            0
program_name                0
degree_level                0
duration_years              0
annual_tuition_usd          0
enrollment_capacity         0
accredited                  0
languages_of_instruction    0
established_year            0
in_demand_field             0
dtype: int64

Unique Universities:
23

Program Categories:
program_category
IT/Computer Science    24
Healthcare             21
Arts/Humanities        21
Tourism/Hospitality    19
Education              18
Engineering            17
Agriculture            14
Business               14
Architecture           11
Law                    11
Name: count, dtype: int64


## 2. Data Cleaning & Transformation

### 2.1 Extract Unique Companies

In [6]:
# Extract unique companies
companies_df = jobs_df[['company_name', 'industry']].drop_duplicates()
companies_df = companies_df.dropna(subset=['company_name'])

print(f"Found {len(companies_df)} unique companies")
print(f"\nTop 10 Industries:")
print(companies_df['industry'].value_counts().head(10))

Found 110 unique companies

Top 10 Industries:
industry
IT/Software            13
Banking/Finance        11
Hospitality/Tourism    10
Transportation          8
Healthcare              8
NGO/Non-Profit          8
Real Estate             7
Retail                  7
Media/Advertising       7
Manufacturing           7
Name: count, dtype: int64


### 2.2 Extract & Normalize Skills

In [7]:
# Extract all skills from jobs
all_skills = set()

for skills_str in jobs_df['required_skills'].dropna():
    skills = [s.strip() for s in str(skills_str).split(',')]
    all_skills.update(skills)

# Remove empty strings
all_skills = {s for s in all_skills if s}

print(f"Found {len(all_skills)} unique skills")
print(f"\nSample skills: {list(all_skills)[:20]}")

Found 32 unique skills

Sample skills: ['React', 'Driving License', 'Customer Service', 'Teamwork', 'Adobe Illustrator', 'HTML/CSS', 'Excel', 'Video Editing', 'Khmer', 'English', 'Data Analysis', 'PowerPoint', 'Communication', 'Java', 'Chinese', 'Photography', 'French', 'Adobe Photoshop', 'Microsoft Office', 'Leadership']


In [9]:
# Categorize skills into Technical, Soft, or Language

LANGUAGE_SKILLS = {'Khmer', 'English', 'Chinese', 'French', 'Japanese', 'Korean', 'Vietnamese'}

SOFT_SKILLS = {
    'Communication', 'Teamwork', 'Leadership', 'Problem Solving', 
    'Time Management', 'Customer Service', 'Project Management',
    'Accounting', 'Photography'
}

def categorize_skill(skill_name):
    if skill_name in LANGUAGE_SKILLS:
        return SkillType.LANGUAGE
    elif skill_name in SOFT_SKILLS:
        return SkillType.SOFT
    else:
        return SkillType.TECHNICAL

# Create skills dataframe
skills_df = pd.DataFrame([
    {'name': skill, 'skill_type': categorize_skill(skill)} 
    for skill in all_skills
])

print("Skills by Type:")
print(skills_df['skill_type'].value_counts())
print(f"\nSample Technical Skills: {skills_df[skills_df['skill_type'] == SkillType.TECHNICAL]['name'].head(10).tolist()}")
print(f"\nLanguage Skills: {skills_df[skills_df['skill_type'] == SkillType.LANGUAGE]['name'].tolist()}")

Skills by Type:
skill_type
SkillType.TECHNICAL    19
SkillType.SOFT          9
SkillType.LANGUAGE      4
Name: count, dtype: int64

Sample Technical Skills: ['React', 'Driving License', 'Adobe Illustrator', 'HTML/CSS', 'Excel', 'Video Editing', 'Data Analysis', 'PowerPoint', 'Java', 'Adobe Photoshop']

Language Skills: ['Khmer', 'English', 'Chinese', 'French']


### 2.3 Extract Unique Universities

In [10]:
# Extract unique universities
unis_df = universities_df[['university_name', 'university_type', 'location', 'established_year']].drop_duplicates()

print(f"Found {len(unis_df)} unique universities")
print(f"\nPublic vs Private:")
print(unis_df['university_type'].value_counts())
print(f"\nLocations:")
print(unis_df['location'].value_counts())

Found 161 unique universities

Public vs Private:
university_type
Public     112
Private     49
Name: count, dtype: int64

Locations:
location
Phnom Penh       109
Battambang        15
Sihanoukville      8
Svay Rieng         8
Kampong Thom       8
Kampong Cham       7
Siem Reap          6
Name: count, dtype: int64


### 2.4 Clean Job Data

In [11]:
# Clean and prepare jobs data
jobs_clean = jobs_df.copy()

# Convert posted_date to datetime
jobs_clean['posted_date'] = pd.to_datetime(jobs_clean['posted_date'], errors='coerce')

# Handle missing salaries (set to 0 or median)
jobs_clean['salary_min_usd'] = jobs_clean['salary_min_usd'].fillna(0)
jobs_clean['salary_max_usd'] = jobs_clean['salary_max_usd'].fillna(0)

print("Jobs data cleaned")
print(f"Date range: {jobs_clean['posted_date'].min()} to {jobs_clean['posted_date'].max()}")
print(f"Salary range: ${jobs_clean['salary_min_usd'].min():.0f} - ${jobs_clean['salary_max_usd'].max():.0f}")

Jobs data cleaned
Date range: 2025-08-12 00:00:00 to 2025-12-09 00:00:00
Salary range: $150 - $9999


## 3. Data Seeding to Database

### 3.1 Seed Companies

In [ ]:
with app.app_context():
    # Seed Companies
    print("🌱 Seeding Companies...")
    company_map = {}  # Map company name to Company object
    
    for _, row in companies_df.iterrows():
        # Check if company already exists
        existing = Company.query.filter_by(name=row['company_name']).first()
        if existing:
            company_map[row['company_name']] = existing
            continue
        
        company = Company(
            name=row['company_name'],
            industry=row['industry']
        )
        db.session.add(company)
        company_map[row['company_name']] = company
    
    db.session.commit()
    print(f"Processed {len(company_map)} companies (new + existing)")

 Seeding Companies...


IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "ix_companies_name"
DETAIL:  Key (name)=(Cambodia Angkor Air) already exists.

[SQL: INSERT INTO companies (name, industry, created_at) SELECT p0::VARCHAR, p1::VARCHAR, p2::TIMESTAMP WITHOUT TIME ZONE FROM (VALUES (%(name__0)s, %(industry__0)s, %(created_at__0)s, 0), (%(name__1)s, %(industry__1)s, %(created_at__1)s, 1), (%(name__2)s, ... 6261 characters truncated ...  imp_sen(p0, p1, p2, sen_counter) ORDER BY sen_counter RETURNING companies.id, companies.id AS id__1]
[parameters: {'name__0': 'China State Construction Engineering Corporation', 'industry__0': 'Construction', 'created_at__0': datetime.datetime(2025, 12, 11, 3, 23, 50, 796737), 'name__1': 'Chip Mong Land', 'industry__1': 'Real Estate', 'created_at__1': datetime.datetime(2025, 12, 11, 3, 23, 50, 796742), 'name__2': 'Sofitel Phnom Penh Phokeethra', 'industry__2': 'Hospitality/Tourism', 'created_at__2': datetime.datetime(2025, 12,

### 3.2 Seed Skills

In [ ]:
with app.app_context():
    print("🌱 Seeding Skills...")
    skill_map = {}  # Map skill name to Skill object
    
    for _, row in skills_df.iterrows():
        # Check if skill already exists
        existing = Skill.query.filter_by(name=row['name']).first()
        if existing:
            skill_map[row['name']] = existing
            continue
        
        skill = Skill(
            name=row['name'],
            skill_type=row['skill_type']
        )
        db.session.add(skill)
        skill_map[row['name']] = skill
    
    db.session.commit()
    print(f"✅ Processed {len(skill_map)} skills (new + existing)")

### 3.3 Seed Jobs with Skills

In [ ]:
with app.app_context():
    print("🌱 Seeding Jobs...")
    
    # Reload company and skill maps from database
    company_map = {c.name: c for c in Company.query.all()}
    skill_map = {s.name: s for s in Skill.query.all()}
    
    count = 0
    skipped = 0
    for _, row in jobs_clean.iterrows():
        # Check if job already exists
        if Job.query.filter_by(id=row['job_id']).first():
            skipped += 1
            continue
        
        # Get company
        company = company_map.get(row['company_name'])
        if not company:
            continue
        
        # Create job
        job = Job(
            id=row['job_id'],
            title=row['job_title'],
            company_id=company.id,
            location=row['location'],
            employment_type=row['employment_type'],
            experience_level=row['experience_level'],
            salary_min_usd=row['salary_min_usd'],
            salary_max_usd=row['salary_max_usd'],
            degree_required=row['degree_required'],
            languages_required=row['languages_required'],
            description=row['description'],
            posted_date=row['posted_date'],
            is_active=row['is_active']
        )
        
        # Add skills
        if pd.notna(row['required_skills']):
            skill_names = [s.strip() for s in str(row['required_skills']).split(',')]
            for skill_name in skill_names:
                skill = skill_map.get(skill_name)
                if skill:
                    job.skills.append(skill)
        
        db.session.add(job)
        count += 1
        
        # Commit in batches
        if count % 100 == 0:
            db.session.commit()
            print(f"  Processed {count} jobs...")
    
    db.session.commit()
    print(f"✅ Seeded {count} new jobs (skipped {skipped} existing)")

### 3.4 Seed Universities

In [ ]:
with app.app_context():
    print("🌱 Seeding Universities...")
    university_map = {}  # Map university name to University object
    
    for _, row in unis_df.iterrows():
        # Check if university already exists
        existing = University.query.filter_by(name=row['university_name']).first()
        if existing:
            university_map[row['university_name']] = existing
            continue
        
        university = University(
            name=row['university_name'],
            university_type=row['university_type'],
            location=row['location'],
            established_year=row['established_year']
        )
        db.session.add(university)
        university_map[row['university_name']] = university
    
    db.session.commit()
    print(f"✅ Processed {len(university_map)} universities (new + existing)")

### 3.5 Seed Programs

In [ ]:
with app.app_context():
    print("🌱 Seeding Programs...")
    
    # Reload university map from database
    university_map = {u.name: u for u in University.query.all()}
    
    count = 0
    skipped = 0
    for _, row in universities_df.iterrows():
        # Check if program already exists
        if Program.query.filter_by(id=row['university_id']).first():
            skipped += 1
            continue
        
        university = university_map.get(row['university_name'])
        if not university:
            continue
        
        program = Program(
            id=row['university_id'],
            university_id=university.id,
            program_name=row['program_name'],
            program_category=row['program_category'],
            degree_level=row['degree_level'],
            duration_years=row['duration_years'],
            annual_tuition_usd=row['annual_tuition_usd'],
            enrollment_capacity=row['enrollment_capacity'],
            accredited=row['accredited'],
            languages_of_instruction=row['languages_of_instruction'],
            in_demand_field=row['in_demand_field']
        )
        
        db.session.add(program)
        count += 1
    
    db.session.commit()
    print(f"✅ Seeded {count} new programs (skipped {skipped} existing)")

## 4. Verification & Analytics

### 4.1 Verify Data Counts

In [ ]:
with app.app_context():
    print("📊 Database Summary:")
    print(f"  Companies: {Company.query.count()}")
    print(f"  Jobs: {Job.query.count()}")
    print(f"  Skills: {Skill.query.count()}")
    print(f"  Universities: {University.query.count()}")
    print(f"  Programs: {Program.query.count()}")

### 4.2 Sample Queries

In [ ]:
with app.app_context():
    # Top 10 most in-demand skills
    print("🔥 Top 10 Most In-Demand Skills:")
    skills = Skill.query.all()
    skill_counts = [(s.name, len(s.jobs)) for s in skills]
    skill_counts.sort(key=lambda x: x[1], reverse=True)
    for skill, count in skill_counts[:10]:
        print(f"  {skill}: {count} jobs")

In [ ]:
with app.app_context():
    # Jobs by location
    print("\n📍 Jobs by Location:")
    from sqlalchemy import func
    location_counts = db.session.query(
        Job.location, 
        func.count(Job.id)
    ).group_by(Job.location).order_by(func.count(Job.id).desc()).limit(10).all()
    
    for location, count in location_counts:
        print(f"  {location}: {count} jobs")

In [ ]:
with app.app_context():
    # Average salary by experience level
    print("\n💰 Average Salary by Experience Level:")
    from sqlalchemy import func
    salary_by_level = db.session.query(
        Job.experience_level,
        func.avg((Job.salary_min_usd + Job.salary_max_usd) / 2).label('avg_salary')
    ).filter(Job.salary_min_usd > 0).group_by(Job.experience_level).all()
    
    for level, avg_salary in salary_by_level:
        print(f"  {level}: ${avg_salary:.2f}")